In [ ]:
!pip install PyPDF2


In [ ]:
import os
import json
import re
import PyPDF2

# Function to extract PDFs from a folder
def extract_pdfs_from_folder(pdf_folder):
    pdf_files = []
    for file_name in os.listdir(pdf_folder):
        if file_name.endswith('.pdf'):
            pdf_files.append(os.path.join(pdf_folder, file_name))
    return pdf_files

# Robust title extractor with multi-line handling and city/date filtering
def extract_title_from_pdf(pdf_text):
    lines = pdf_text.splitlines()

    skip_patterns = [
        r'DFRWS.*\d{4}',
        r'Digital Investigation',
        r'Forensic Science International',
        r'ScienceDirect',
        r'ELSEVIER',
        r'CrossMark',
        r'Contents lists available at',
        r'journal homepage',
        r'Available at',
        r'www\.',
        r'^$',
        r'^DIGITAL FORENSIC RESEARCH CONFERENCE$',
        r'^DOI[:\s]',
        r'^http[s]?://',
        r'creativecommons.org',
    ]
    bad_title_patterns = [
        r'^\(?[A-Z][a-z]+, [A-Z]{2} \(',       # City, State ( e.g., Syracuse, NY (
        r'^\(?[A-Z][a-z]+ \d{1,2}(st|nd|rd|th)?\)?',  # Aug 6th
        r'^\(?\d{4}\)?$',                      # year
    ]
    stop_patterns = [
        r'^By ',
        r'From the proceedings of',
        r'DFRWS \d{4}',
        r'\(\w{3,9} \d{1,2}',
        r'^\d{4} \d{1,2}:\d{2}',               # timestamps
        r'^Elsevier',
        r'^Keywords',
    ]

    clean_lines = [line.strip() for line in lines if line.strip()]
    title_candidates = []

    for i in range(min(50, len(clean_lines))):
        line = clean_lines[i]
        if any(re.search(pat, line, re.IGNORECASE) for pat in skip_patterns):
            continue
        if any(re.search(pat, line.strip(), re.IGNORECASE) for pat in bad_title_patterns):
            continue
        if any(re.search(pat, line, re.IGNORECASE) for pat in stop_patterns):
            break
        if len(line.split()) < 2:
            continue
        title_candidates.append((i, line))

    # Step 1: Multi-line title merge
    for i, line in title_candidates:
        if i + 1 < len(clean_lines):
            next_line = clean_lines[i + 1].strip()
            full_title = f"{line} {next_line}"
            if 6 <= len(full_title.split()) <= 20 and not full_title.endswith('.'):
                return full_title

    # Step 2: Title right before "By"
    for idx, line in title_candidates:
        if idx + 1 < len(clean_lines) and clean_lines[idx + 1].lower().startswith("by "):
            return line

    # Step 3: Fallback to any decent candidate
    for _, line in title_candidates:
        if len(line.split()) >= 4:
            return line

    return "Unknown Title"

# Function to extract text from a PDF
def extract_text_from_pdf(pdf_file):
    try:
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        pdf_text = ""
        for page_num in range(len(pdf_reader.pages)):
            page_text = pdf_reader.pages[page_num].extract_text()
            if page_text:
                pdf_text += page_text
        return pdf_text
    except KeyError as e:
        print(f"Error processing {pdf_file}: {e}")
        return None
    except Exception as e:
        print(f"Unexpected error processing {pdf_file}: {e}")
        return None

# Main script
pdf_folder = r"full_dataset"
output_path = 'extracted_test_papers_new_dfrws.jsonl'

papers = []
pdf_files = extract_pdfs_from_folder(pdf_folder)

for pdf_file in pdf_files:
    text = extract_text_from_pdf(pdf_file)
    if text:
        title = extract_title_from_pdf(text)
        papers.append({"title": title, "content": text})
        print(f" {os.path.basename(pdf_file)} → Title: {title}")
    else:
        print(f" Failed to extract text from: {os.path.basename(pdf_file)}")

# Save output to JSONL
with open(output_path, 'w', encoding='utf-8') as f:
    for paper in papers:
        safe_text = json.dumps(paper, ensure_ascii=False).encode('utf-8', 'ignore').decode('utf-8')
        f.write(safe_text + "\n")

print(f"\n Extraction complete. Output saved to: {output_path}")


In [ ]:
!pip install python-dotenv
from openai import OpenAI
import os
from dotenv import load_dotenv

Tools

In [43]:
def generate_title_prompt(paper):
    title = paper['title']
    content = paper['content']
    return f'''
    You are tasked with extracting the full title from the digital forensics paper titled "{title}".

    Guidelines:
    - The title is usually at the top of the first page or in the first section.
    - Extract the title in its entirety.

    Your response must be in the following JSON format:
    {{
        "title": "Title of the paper here"
    }}

    Here is the paper content:
    <Start of Paper Content>
    {content}
    <End of Paper Content>

    Your response: """
    '''


def generate_tools_prompt(paper):
    title = paper['title']
    content = paper['content']
    
    return f'''
    You are tasked with extracting **tools** mentioned in the digital forensics paper titled "{title}".

   Guidelines:
    \t1. A **tool** is any named software, framework, script, or system used or created for forensic or anti-forensic purposes.
    \t- Only include a tool if it is actually used, created, or extended in the paper. 
    \t- If a tool is **referenced** or just **mentioned** for context and NOT used, do NOT include it in the tools output. For instance, referred in  
    sections like **Related Work** or **Literature Review** section does not count.

    \t2. For each tool actually **used**, **created**, or **extended**, provide:
       \t- "tool_name"
       \t- "action": "used" or "created" or "extended"
       \t- "repository_link" (URL or empty string)
       \t- "license": must be one of:
          \t\t- "open-source": source code is publicly available, allowing others to inspect, modify, and extend. Open-source tools are continuously updated and widely reused. They enable creativity and expansion since others can build upon the original code. Examples: Kali Linux, CAINE, Autopsy.
          \t\t- "proprietary": source code is closed and controlled by the originator (e.g., company or vendor). Only the developer can modify or distribute it. Proprietary tools are widely used in practice, especially in law enforcement, but cannot be extended by the community. Examples: FTK Forensic Toolkit, FTK Imager, Magnet AXIOM (Magnet Forensics).
          \t\t- "not-mentioned": if the license type is not explicitly stated and no reliable source (e.g., URL, DOI) is available.
       \t- "origin": one of:
          \t\t- "academic_research_DFRWS" if the tool was first introduced by this DFRWS paper
          \t\t- "academic_research_external" if the tool was created in other academic venues (conferences, journals, academic projects)
          \t\t- "organization" if the tool was created by companies, vendors, or non-academic organizations
          \t\t- "not-mentioned" if the origin is not explicitly stated

    \t3. Special case:
       \t- If the paper introduces a plugin, module, extension, or significant modification of an existing tool, mark "action" as "extended" and apply the same origin rules.
       \t- Also list the base tool separately if it was explicitly used.

    \t4. If the paper uses **no tools**, return: null

       {{
         "tools": [{{"tool_name": null, "action": null, "repository_link": null, "repository_type": null, "authorship": null}}]
       }}

    \t- Null Cases Example 1: In the paper "How to Reuse Knowledge about Forensic Investigations, the authors didnt used/created/extended any tool, so the output should be "null" in all fields.

    {{
         "tools": [{{"tool_name": null, "action": null, "repository_link": null, "repository_type": null, "authorship": null}}]
    }}

    \t- Null Cases Example 2: In the paper "A survey of forensic characterization methods for physical devices", they author didnt used/created/extended since its a survey paper, so the output should be:
    
    {{
         "tools": [{{"tool_name": null, "action": null, "repository_link": null, "repository_type": null, "authorship": null}}]
    }}
    
    \t5. If a tool is found but any field (action, repository_link, repository_type, or authorship) is not mentioned, assign "not-mentioned" to that field.

    JSON Format:
    {{
      "tools": [
        {{
          "tool_name": "Volatility",
          "action": "used",
          "repository_link": "https://github.com/volatilityfoundation/volatility",
          "repository_type": "open-source",
          "origin": "organization"
        }},
    
        {{
          "tool_name":"DROP",
          "action": "created"
          "repository_link": "https://github.com/unhcfreg/DROP",
          "repository_type": "open-source",
          "origin": "academic_research_DFRWS"
        }}
      ]
    }}
    \t. For example in the paper "So fresh, so clean: Cloud forensic analysis of the Amazon iRobot Roomba vacuum", DFRWS authors created
    a tool and made it open-source:
    {{
      "tools": [
        {{
          "tool_name": "PyRoomba",
          "action": "created",
          "repository_link": "https://github.com/BiTLab-BaggiliTruthLab/PyRoomba",
          "repository_type": "open-source",
          "origin": "academic_research_DFRWS"
        }}
    }}
    
    Here is the paper content: 
    <Start of Paper Content>
    {content}
    <End of Paper Content>

    Your response: """
    '''


In [ ]:
import os
import csv
from openai import OpenAI
from dotenv import load_dotenv

# Load API key from .env
load_dotenv("api_key.env")
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# Your prompt generators must be defined above or in the same file:
# def generate_title_prompt(paper): …
# def generate_tools_prompt(paper): …

# Your processor
def process_papers_for_tasks(papers, tasks):
    task_results = {}
    for i, paper in enumerate(papers):
        paper_title = paper['title']
        print(f"\nProcessing paper: {paper_title}")
        task_results[paper_title] = {}

        for task in tasks:
            if task == "title":
                user_prompt = generate_title_prompt(paper)
            elif task == "tools":
                user_prompt = generate_tools_prompt(paper)
            else:
                continue

            try:
                response = client.chat.completions.create(
                    model='gpt-4o-mini',
                    messages=[{"role": "user", "content": user_prompt}],
                    temperature=0.2,
                    max_tokens=5000
                )
                response_text = response.choices[0].message.content
                print(response_text)
                task_results[paper_title][task] = response_text

            except Exception as e:
                print(f"Error processing {task} for paper {i+1}: {e}")
                task_results[paper_title][task] = f"error: {e}"

    return task_results


# Run on all papers

test_papers = papers  # assumes 'papers' is a list of dicts with 'title' and 'content'
tasks = ["title", "tools"]

results = process_papers_for_tasks(test_papers, tasks)

# Save CSV
with open("results_new_last_tools.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["Paper Title"] + tasks)
    for paper_title, r in results.items():
        row = [paper_title] + [r.get(task, "No result") for task in tasks]
        writer.writerow(row)

print("Results saved to results_new_last_tools.csv")


Metadata Extraction

In [ ]:
def generate_all_metadata_prompt(task, paper, ontology_json=None):
    title = paper['title']
    content = paper['content']

    if task == "title":
        return f'''
        You are tasked with extracting the full title from the digital forensics paper titled "{title}".

        Guidelines:
        - The title is usually at the top of the first page or in the first section.
        - Extract the title in its entirety.

        Your response must be in the following JSON format:
        {{
            "title": "Title of the paper here"
        }}

        Here is the paper content:
        <Start of Paper Content>
        {content}
        <End of Paper Content>

        Your response: """
        '''

    elif task == "authors":
        return f'''
        You are tasked with extracting the list of authors from the paper titled "{title}".

        Guidelines:
        - The authors' names are typically listed directly below the title or in the header/footer.
        - Extract all authors, separated by commas.

        Your response must be in the following JSON format:
        {{
            "authors": "Comma-separated list of authors' names here"
        }}

        Here is the paper content:
        <Start of Paper Content>
        {content}
        <End of Paper Content>

        Your response: """
        '''
    elif task == "school_names":
        return f'''
        You are tasked with extracting the name(s) of academic institutions or organizations affiliated with the authors of the paper titled "{title}".

        Guidelines:
        - The school or institutional affiliations are usually listed below the authors' names or in the first page.
        - Extract **all** school names or affiliations mentioned.
        - Include universities, colleges, research institutions, or companies if provided.
        - Remove duplicates if the same institution is mentioned multiple times.

        Your response must be returned in the following JSON format:
        {{
            "school_names": ["University of California, Berkeley", "National University of Singapore"]
        }}

        Here is the paper content:
        <Start of Paper Content>
        {content}
        <End of Paper Content>

        Your response: """
        '''
    elif task == "author_countries":
        return f'''
        You are tasked with extracting the country or countries where the authors or their affiliated institutions are based for the paper titled "{title}".

        Guidelines:
        - Country names are often listed after the institution name, or appear in the corresponding author information, address section, or footnotes.
        - Only look at the countries mentioned alongside author names, typically found at the top of the paper. Do not scan the entire document.
        - If a country is not explicitly stated but can be reliably inferred from a well-known institution (e.g., MIT → USA), include it.
        - Do not guess or hallucinate. If the country cannot be determined, return "null".

        Your response must be in the following JSON format:
        {{
            "author_countries": ["USA", "Germany"]
        }}

        Here is the paper content:
        <Start of Paper Content>
        {content}
        <End of Paper Content>

        Your response: """
        '''


    elif task == "conference":
        return f'''
        
        You are tasked with identifying the conference of publication for the paper titled "{title}".

        Guidelines:
        \t1- The conference name will always be one of the following:
        \t- "DFRWS USA"
        \t- "DFRWS Europe"
        \t- "DFRWS APAC"
        
        \t2- Read the paper carefully to determine which DFRWS conference it was published in.
        
        Your response must be in the following JSON format:
        {{
            "conference": "DFRWS USA"
        }}

        Here is the paper content:
        <Start of Paper Content>
        {content}
        <End of Paper Content>

        Your response: """
        '''

    elif task == "published_year":
        return f'''
        You are tasked with extracting the year of publication from the digital forensics paper titled "{title}".

        Guidelines:
        - The year is usually found near the conference name or in the paper's footer/header.

        Your response must be in the following JSON format:
        {{
            "year": "Year of publication here"
        }}

        Here is the paper content:
        <Start of Paper Content>
        {content}
        <End of Paper Content>

        Your response: """
        '''
    elif task == "forensic_vs_anti":
        return f'''
       
       You are tasked with extracting the following metadata from the provided digital forensics paper titled "{title}":
       
       You must classify the paper as either **Forensic** or **Anti-Forensic**.

        Definition for Identifying paper type:
        \t1- **Digital Forensic**: A paper that supports, enhances, or proposes solutions for conducting digital forensic investigations.
        \t2- **Anti-Forensic**: A paper that explores techniques or tools meant to subvert, obscure, or defeat digital forensic processes, often discussed either to develop countermeasures or expose threats.

        Book Definitions:
        \t1- **Digital Forensics**: The application of science to the identification, collection/acquisition, examination, and analysis of data while preserving the integrity of the information and maintaining a strict chain 
        of custody for the data.
        \t2- **Anti-Forensic**: A technique for concealing or destroying data so that others cannot access it.

        Guidelines:
        \t1- Determine whether the focus of the paper aligns more with digital forensic goals (acquisition, analysis, etc.) or with undermining such goals.
        \t2- You must assess the paper’s core contribution and stance. Just because a paper discusses anti-forensic techniques does **not** necessarily mean it is an anti-forensic paper.
        \t3- If a paper studies anti-forensic techniques with the goal of developing better forensic tools, defenses, or acquisition methods, it is still considered ** Digital Forensic**.
        \t4- If a paper introduces or promotes anti-forensic techniques without a forensic enhancement objective, then it is **Anti-Forensic**.


        Your response must be in the following JSON format:
        {{
            "forensic_type": "Digital Forensic"  // or "Anti-Forensic"
        }}
    
    
        \t- Examples in JSON format.
        Example 1:
        In the paper "Anti-forensic resilient memory acquisition", the authors design a memory acquisition technique that is robust against anti-forensic attacks.
        Since the paper contributes to strengthening digital forensic methods and improving resilience, it should be classified as forensic.
        {{
            "forensic_type": "Forensic"
        }}

        Example 2:
        In the paper "Android anti-forensics through a local paradigm", the authors present practical techniques to undermine forensic processes on Android devices, including delaying and manipulating evidence. 
        These methods aim to defeat forensic tools, making the paper anti-forensic.
        {{
            "forensic_type": "Anti-Forensic"
        }}

        Example 3:
        In the paper "Forensic carving of network packets and associated data structures" (2011), the focus is on developing techniques for reconstructing network packets from raw data to aid in forensic investigations. 
        It enhances forensic analysis and does not attempt to conceal or disrupt it. Thus, it is classified as:
        {{
            "forensic_type": "Forensic"
        }}


        Here is the paper content:
        <Start of Paper Content>
        {content}
        <End of Paper Content>

        Your response: """
        '''
    
    elif task == "ontology_classification":
            return f'''
            
            You are tasked with classifying the research paper titled "{title}" based on a structured digital forensics ontology.
            Guidelines:
            \t1- Choose only one value for each of the following fields: discipline, subdiscipline, object, subobject.
            \t2- If no match is found, return "unknown" for that field.
            \t3- If a new domain or object/subobject not in the list is clearly found, return it directly.
            \t4- You MUST choose only from the taxonomy structure below:

            {{
                "Computer Forensics": {{
                "Server Forensics": {{"Hard Disk": ["Logs"], "Registers": [], "RAM": []}},
                "Laptop Forensics": {{"Hard Disk": ["Logs"], "Registers": [], "RAM": []}},
                "Desktop Forensics": {{"Hard Disk": ["Logs"], "Registers": [], "RAM": []}}
            }},
                "Software Forensics": {{
                "Operating System Forensics": {{"File systems for Windows/Mac/Unix/Linux": [], "Windows/Mac/Unix/Linux": []}},
                "Application Software Forensics": {{"Mail Services": [], "Web Services": [],  "DBMS": [], "Access Control Systems": ["Building security Logs", "Passport control Logs"], "E-Commerce Services": ["Credit Card Logs", "Bank Logs", "E-payment Logs", "WebShop Logs"]}},
                "Forensic Tools Analysis (Open source/Proprietary)": {{"E4Case/FTK/File Hound/Sleuthkit/WinHex": []}}
            }},
                "Database Forensics": {{
                "Database Metadata/Contents Forensics": {{"DBMS": [], "Databases": []}}
            }},
                "Multimedia Forensics": {{
                "Image Forensics": {{"Digital Images": []}},
                "Video Forensics": {{"Digital Video": []}},
                "Audio Forensics": {{"Digital Audio": []}}
            }},
                "Device Forensics": {{
                "Peripherals Device Forensics": {{"Copiers": [], "Printers": [], "Scanners": []}}
                "Network Enabled Device Forensics": {{"Wireless AP": [], "IDS": [], "Firewalls": [], "Hubs": [], "Switches": [], "Routers": []}},
                "Storage Device Forensics": {{"RFID Tags/Smart cards/Memory cards": [], "DVD/CD/Floppy/Tapes": [], "External Hard Drives": [], "Thumb Drive": [], "Digital Music Players": ["iPod"]}},
                "Large Scale Device Forensics": {{"SAN (Storage Area Network)": [], "NAS (Network Attached Storage)": []}},
                "Obscure Device Forensics": {{"Recording Devices (Video/Audio)": ["Camcorder", "surveillance camera"], "Gaming Devices": ["Play Station", "Xbox", "PSP"]}},
                "Mobile Forensics": {{"PDAs": ["BlackBerry"], "Smart/Cell phones": ["Phone Memory Cards", "SIM Cards"], "Tablets": []}}
                "Small Scale Device Forensics": {{ "Embedded Devices": [], "GPS Devices": []}}
            }},
                "Network Forensics": {{
                "Cloud Forensics": {{"Clouds (Cloud Computing)": []}},
                "Telecom Network Forensics": {{"Cell Phone/Telecom Service Provider Network": []}},
                "Internet Forensics": {{"Web Documents": [], "Webmails": [], "Emails": [], "Domain Name Records": [], "ISP Logs": []}},
                "Wireless Forensics": {{"Bluetooth, Infrared, Wi-Fi": []}}
            }},
                "IoT Forensics": {{
                "Smart Home Devices": {{"IoT Cameras": [], "Thermostats": [], "Relays / Switch Actuators":[]}},
                "Industrial IoT Systems": {{"SCADA": [], "ICS Logs": []}},
                "Drone Forensic":{{"Drone (UAVs)": [" Flight Logs", "GPS Data", "Video Footage", "Sensor Data", "Communication Logs"]}},
                "Embedded IoT Devices": {{
                  "Operating Systems": ["Contiki / Contiki-NG", "RIOT", "TinyOS", "FreeRTOS"],
                  "File Systems": ["Coffee", "YAFFS2", "LittleFS", "JFFS2", "FAT"],
                  "Memory Artifacts": ["Flash Dumps", "Micrologs", "Wear-Leveling Metadata"],
                  "Device Types": ["Motes (Sky, Z1)", "Sensor Nodes", "MCU Boards (CC13xx/26xx, CC2538)"]
                }}
            }},
                "AI Forensics": {{
                "AI Training Forensics": {{"Training Process Forensics": ["Training Chain of Custody", "Objective / Cost Function Forensics", "Pre-processing Forensics"], "Dataset Forensics": ["Collection Chain of Custody", "Labeling Chain of Custody", "QA Forensics"],"Environment Forensics": []}},
                "AI Substrate Forensics": {{"Disk, Network, Sensor, Actuator": [] }},
                "AI Application Forensics": {{"API, Artifacts": []}},
                "AI Model Forensics": {{"Model Authentication Forensics, Model Fingerprinting / Ballistics Forensics, Model Identification / Extraction Forensics, Model Performance Forensics, Model Malware Forensics, Model Chain of Custody": []}}
            }},
                "Blockchain Forensics": {{
                "Wallet Forensics": {{"Cryptocurrency Wallets": ["Public/Private Keys"]}},
                "Transaction Analysis": {{"Blockchain Ledger": ["Smart Contracts", "Hashes"]}}
            }},
               "Knowledge Systematization": {{
                "Systematization of Knowledge (SoK)": [],
                "Systematic Literature Review (SLR)": {{"SLR Papers": [], "Comparative Studies": []}},
                "Ontology Development/Taxonomy Development": {{"Ontologies": [], "Taxonomies": []}},
                "Survey Papers": [],
                "Frameworks": {{"Validation Methodologies": [], "Community Standards": []}},
                "Methodological Awareness / Risk Analysis": []
            }}
        }}

            Your response must be in the following JSON format:
            {{
                "discipline": "...",
                "subdiscipline": "...",
                "object": "...",
                "subobject": "..."
            }}
            
            \t5- Always assign the discipline and subdiscipline based on where the potential digital evidence is primarily collected 
            and how it is logically stored or accessed.
            \t-If the evidence is collected through physical acquisition (e.g., raw disk sectors, hardware memory dumps, chip-off), classify it under
            **Computer Forensics** with the appropriate subdiscipline **Laptop/Desktop/Server** and objects (Disk, RAM, Registers, etc.).
            \t-If the evidence is collected through an operating system, software, or protocol layer (e.g., file system metadata, OS logs, 
            applications, network flows, cloud APIs), classify it under **Software Forensics**or **Network Forensics**, depending on the context.

            \t6- Examples:

            \t- In the paper *“Avoiding Burnout at the Digital Forensics Coalface”*, the authors perform a thematic synthesis 
            and propose evidence-based frameworks for stress management in the digital forensics workforce.

            {{
                "discipline": "Knowledge Systematization",
                "subdiscipline": "Frameworks",
                "object": "Validation Methodologies",
                "subobject": ""
            }}
            
            \t- Another papers like the one that only propose standardized validation frameworks, test guidelines, or 
            community standards (e.g., Testing Disk Imaging Tools return **discipline** as **Knowledge Systematization**
            under subdiscipline **Frameworks**.

            \t- There are multiple papers in our dataset that falls under **AI Forensics** please handle with care. 
            For AI forensics, classify each paper into one of the four AI Forensics subdisciplines based on its main focus. 
            Always base your decision on where the forensic data is acquired. If the data comes from training logs, datasets, or training environments, 
            classify it under **AI Training Forensics**. If it involves analyzing the AI model itself for tampering, theft, or authenticity, choose **AI Model Forensics**.
            If the data is extracted from the hardware, embedded systems, memory, or sensors, assign it to **AI Substrate Forensics**. 
            If the focus is on logs, APIs, outputs, or user-facing artifacts generated during AI use, 
            then it belongs to **AI Application Forensics**.

            The paper **ChatGPT for digital forensic investigation: The good, the bad, and the unknown** clearly fits into the AI Forensics ontology, here's the proper classification:
            
            {{
                "discipline": "AI Forensics",
                "subdiscipline": "AI Application Forensics",
                "object": "API, Artifacts",
                "subobject": ""

            }}



            Here is the paper content:
            <Start of Paper Content>
            {content}
            <End of Paper Content>

            Your response: """
            '''
   
    else:
        raise ValueError("Invalid task")

        

In [ ]:
import os
import csv
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv

# Load API key from .env
load_dotenv("api_key.env")

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# Your prompt generator import or definition:
# Make sure generate_all_metadata_prompt is defined exactly as you have it

# Example: 
# from your_prompt_file import generate_all_metadata_prompt

# Main processor
def process_papers_for_tasks(papers, tasks):
    task_results = {}

    for i, paper in enumerate(papers):
        paper_title = paper['title']
        print(f"Processing paper: {paper_title}")

        task_results[paper_title] = {}

        for task in tasks:
            user_prompt = generate_all_metadata_prompt(task, paper)

            try:
                response = client.chat.completions.create(
                    model='gpt-4o-mini',
                    messages=[
                        {
                            "role": "user",
                            "content": user_prompt,
                        }
                    ],
                    temperature=0.2,
                    max_tokens=1500
                )

                response_text = response.choices[0].message.content
                print(response_text)
                task_results[paper_title][task] = response_text

            except Exception as e:
                print(f"Error processing {task} for paper {i+1}: {e}")
                task_results[paper_title][task] = "error: " + str(e)

    return task_results

#  Replace 'papers' with your dataset
# For testing, slice first 5 papers
test_papers = papers[:]

#  Define your actual tasks
tasks = ["title", "authors", "school_names", "author_countries", "conference", "published_year", "forensic_vs_anti", "ontology_classification"]

# Process papers
all_results = process_papers_for_tasks(test_papers, tasks)

# Save to CSV exactly like you had it
output_csv_path = 'results_combined_prompts_new.csv'
utput_jsonl_path = 'results_combined_prompts_new.jsonl'
with open(output_csv_path, 'w', newline='', encoding='utf-8') as file:
    csv_writer = csv.writer(file)
    headers = ['Paper Title'] + tasks
    csv_writer.writerow(headers)

    for paper_title, results in all_results.items():
        row = [paper_title]
        for task in tasks:
            row.append(results.get(task, "No result"))
        csv_writer.writerow(row)
# Save to JSONL
with open(output_jsonl, "w", encoding="utf-8") as jsonl_file:
    for title, results in processed_results.items():
        jsonl_file.write(json.dumps({title: results}) + "\n")

print(f"Final results saved to {output_csv} and {output_jsonl}.")



DATA VISUALIZATION

Venue × Discipline table (USA, EU, APAC) — counts + MEDIAN + CV

In [ ]:
# -------------------------------
# 8) Venue × Discipline table (USA, EU, APAC)
# -------------------------------

# Helper: map conference string -> venue bucket
def map_conference_to_venue(conf_str: str) -> str:
    s = (conf_str or "").strip().upper()
    if "USA" in s:
        return "DFRWS USA"
    if "EU" in s or "EUROPE" in s:
        return "DFRWS EU"
    if "APAC" in s or "ASIA" in s or "ASIA-PACIFIC" in s or "ASIA PACIFIC" in s:
        return "DFRWS APAC"
    return "Unknown"

records = []
for _, row in df.iterrows():
    # parse fields
    conf_json = safe_parse_json(row.get("conference", ""))
    ont_json  = safe_parse_json(row.get("ontology_classification", ""))

    conf_raw = conf_json.get("conference", "")
    venue    = map_conference_to_venue(conf_raw)

    discipline = (ont_json.get("discipline", "") or "").strip()

  
    # skip empties & unknown venues
    if not discipline:
        continue
    if venue == "Unknown":
        continue

    records.append({"venue": venue, "discipline": discipline})

venue_disc_df = pd.DataFrame(records)

# Build full matrix, include all venues that have papers and all disciplines seen in your run
venue_disc_matrix = (
    pd.crosstab(venue_disc_df["venue"], venue_disc_df["discipline"])
    .astype(int)
)

# Order columns (disciplines) by global total descending
col_order = venue_disc_matrix.sum(axis=0).sort_values(ascending=False).index
venue_disc_matrix = venue_disc_matrix[col_order]

# Ensure the row order is USA, EU, APAC (only those that exist)
desired_rows = [v for v in ["DFRWS USA", "DFRWS EU", "DFRWS APAC"] if v in venue_disc_matrix.index]
venue_disc_matrix = venue_disc_matrix.reindex(desired_rows)

# Add summary columns
venue_disc_matrix["TOTAL"]   = venue_disc_matrix.sum(axis=1)
venue_disc_matrix["AVERAGE"] = (venue_disc_matrix[col_order].mean(axis=1)).round(2)

print("\n=== Venue × Discipline (counts) ===")
print(venue_disc_matrix)

# Save for your paper/supplement
venue_disc_matrix.to_csv("venue_discipline_matrix.csv")
print("\nSaved: venue_discipline_matrix.csv")


In [ ]:
import numpy as np

disc_cols = [c for c in venue_disc_matrix.columns if c not in ["TOTAL","AVERAGE"]]

# Average across all disciplines (zeros counted) – what you already have
venue_disc_matrix["AVERAGE_ALL"] = (venue_disc_matrix[disc_cols].mean(axis=1)).round(2)

# Average across non-zero disciplines only
venue_disc_matrix["AVERAGE_NONZERO"] = (
    venue_disc_matrix[disc_cols].replace(0, pd.NA).mean(axis=1)
).round(2)

# Median per venue (robust to outliers)
venue_disc_matrix["MEDIAN"] = venue_disc_matrix[disc_cols].median(axis=1).round(2)

# Coefficient of variation (std/mean) – lower = more consistent across domains
means = venue_disc_matrix[disc_cols].mean(axis=1).replace(0, np.nan)
stds  = venue_disc_matrix[disc_cols].std(axis=1, ddof=0)
venue_disc_matrix["CV"] = (stds / means).round(3)

# (Optional) Keep just one "AVERAGE" and rename it:
venue_disc_matrix["AVERAGE"] = venue_disc_matrix["AVERAGE_ALL"].round(2)
venue_disc_matrix.drop(columns=["AVERAGE_ALL"], inplace=True)

# Show full table and save
with pd.option_context("display.max_rows", None, "display.max_columns", None,
                       "display.width", 200, "display.max_colwidth", None):
    print("\n=== Venue × Discipline (counts + averages) ===")
    print(venue_disc_matrix.to_string())

venue_disc_matrix.to_csv("venue_discipline_matrix_with_stats.csv")
print("\nSaved: venue_discipline_matrix_with_stats.csv")


In [ ]:
# -------------------------------
# 8) Venue × Discipline table (USA, EU, APAC) — robust + MEDIAN & CV
# -------------------------------

def map_conference_to_venue(conf_str: str) -> str:
    s = (conf_str or "").strip().upper()
    if "USA" in s:
        return "DFRWS USA"
    if "EU" in s or "EUROPE" in s:
        return "DFRWS EU"
    if "APAC" in s or "ASIA" in s or "ASIA-PACIFIC" in s or "ASIA PACIFIC" in s:
        return "DFRWS APAC"
    return "Unknown"

def extract_conference(row) -> str:
    """
    Try JSON in 'conference'; if that fails, try plain strings / alternate cols.
    """
    # 1) JSON cell like {"conference":"DFRWS USA"}
    conf_json = safe_parse_json(row.get("conference", ""))
    if isinstance(conf_json, dict):
        val = conf_json.get("conference", "") or conf_json.get("name", "")
        if isinstance(val, str) and val.strip():
            return val.strip()

    # 2) If the cell itself is already a plain string like "DFRWS USA"
    raw = row.get("conference", "")
    if isinstance(raw, str) and raw.strip():
        return raw.strip()

    # 3) Fallback column names you might have
    for alt in ("venue", "conf", "event", "conference_name"):
        if alt in row and isinstance(row[alt], str) and row[alt].strip():
            return row[alt].strip()

    return ""

def extract_discipline(row) -> str:
    """
    Prefer a flat 'discipline' column if present; else parse ontology JSON.
    """
    if "discipline" in df.columns:
        val = row.get("discipline", "")
        if isinstance(val, str) and val.strip():
            return val.strip()

    ont_json = safe_parse_json(row.get("ontology_classification", ""))
    if isinstance(ont_json, dict):
        val = (ont_json.get("discipline") or "")
        if isinstance(val, str) and val.strip():
            return val.strip()

    return ""

records = []
debug_counts = {"rows": 0, "kept": 0, "no_conf": 0, "unknown_venue": 0, "no_disc": 0}

for _, row in df.iterrows():
    debug_counts["rows"] += 1

    conf_raw = extract_conference(row)
    venue = map_conference_to_venue(conf_raw)
    disc = extract_discipline(row)


    if not disc:
        debug_counts["no_disc"] += 1
        continue
    if not conf_raw:
        debug_counts["no_conf"] += 1
        continue
    if venue == "Unknown":
        debug_counts["unknown_venue"] += 1
        continue

    records.append({"venue": venue, "discipline": disc})
    debug_counts["kept"] += 1

venue_disc_df = pd.DataFrame.from_records(records, columns=["venue", "discipline"])

# If still empty, fail gracefully with actionable info
if venue_disc_df.empty:
    print("\n[WARN] No venue/discipline records parsed. Debug:", debug_counts)
    print("Tip: Inspect a few raw cells, e.g.:")
    # show a few examples from your dataframe to help verify formats
    print(" - df['conference'].head() =", df.get("conference", pd.Series(dtype=object)).head().to_list())
    print(" - df['ontology_classification'].head() =", df.get("ontology_classification", pd.Series(dtype=object)).head().to_list())
else:
    # Build matrix
    venue_disc_matrix = pd.crosstab(venue_disc_df["venue"], venue_disc_df["discipline"]).astype(int)

    # Order columns by global total (desc)
    col_order = venue_disc_matrix.sum(axis=0).sort_values(ascending=False).index
    venue_disc_matrix = venue_disc_matrix[col_order]

    # Row order: USA, EU, APAC (only those present)
    desired_rows = [v for v in ["DFRWS USA", "DFRWS EU", "DFRWS APAC"] if v in venue_disc_matrix.index]
    venue_disc_matrix = venue_disc_matrix.reindex(desired_rows)

    # ---- Stats (no "non-zero" variants) ----
    core = venue_disc_matrix[col_order]

    venue_disc_matrix["TOTAL"]  = core.sum(axis=1)

    # Median across disciplines (including zeros, per your last preference)
    venue_disc_matrix["MEDIAN"] = core.median(axis=1).round(2)

    # Coefficient of Variation = std/mean (protect against division by zero)
    means = core.mean(axis=1)
    stds  = core.std(axis=1, ddof=1)  # sample std
    cv = (stds / means.replace(0, pd.NA)).fillna(0.0).round(3)
    venue_disc_matrix["CV"] = cv

    print("\n=== Venue × Discipline (counts + MEDIAN & CV) ===")
    print(venue_disc_matrix)

    venue_disc_matrix.to_csv("venue_discipline_matrix.csv")
    print("\nSaved: venue_discipline_matrix.csv")
    print("\n[Debug summary]", debug_counts)


In [ ]:
import pandas as pd
import json
import re
from collections import defaultdict
import matplotlib.pyplot as plt

# -------------------------------
# 1. Load CSV File
# -------------------------------
csv_path = "results_combined_prompts_new.csv"
df = pd.read_csv(csv_path)

# -------------------------------
# 2. Safe JSON Parser
# -------------------------------
def safe_parse_json(cell):
    try:
        cell = re.sub(r"```json|```", "", str(cell)).strip()
        return json.loads(cell)
    except:
        return {}

# -------------------------------
# 3. Extract and Normalize Fields
# -------------------------------
discipline_year_counts = defaultdict(lambda: defaultdict(int))
subdiscipline_year_counts = defaultdict(lambda: defaultdict(int))

for _, row in df.iterrows():
    ont = safe_parse_json(row.get("ontology_classification", ""))
    year_entry = safe_parse_json(row.get("published_year", ""))
    year = year_entry.get("year", None)

    if year:
        try:
            year = int(year)
        except:
            continue

        discipline = ont.get("discipline", "").strip()
        subdiscipline = ont.get("subdiscipline", "").strip()

        # Normalize: Merge Mobile Forensics into Device Forensics
       
        # NEW normalization requested:
        if subdiscipline == "Smart/Cell phones":           #because its an object 
            subdiscipline = "Small Scale Device Forensics"
            
        if discipline:
            discipline_year_counts[discipline][year] += 1
        if subdiscipline:
            subdiscipline_year_counts[subdiscipline][year] += 1

# -------------------------------
# 4. Convert to DataFrames
# -------------------------------
discipline_df = pd.DataFrame(discipline_year_counts).fillna(0).astype(int).sort_index()
subdiscipline_df = pd.DataFrame(subdiscipline_year_counts).fillna(0).astype(int).sort_index()

# -------------------------------
# 5. Plot Line Graph: Discipline Trends
# -------------------------------
plt.figure(figsize=(14, 6))
discipline_df.rolling(window=2, min_periods=1).mean().plot()
plt.title("Discipline Trends in DFRWS (2002–2025)")
plt.xlabel("Year")
plt.ylabel("Number of Papers")
plt.legend(title="Discipline", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# -------------------------------
# 6. Plot Line Graph: Subdiscipline Trends
# -------------------------------
plt.figure(figsize=(14, 6))
subdiscipline_df.rolling(window=2, min_periods=1).mean().plot()
plt.title("Subdiscipline Trends in DFRWS (2002–2025)")
plt.xlabel("Year")
plt.ylabel("Number of Papers")
plt.legend(title="Subdiscipline", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# -------------------------------
# 7. Print and Save Totals
# -------------------------------
discipline_totals = discipline_df.sum().sort_values(ascending=False)
subdiscipline_totals = subdiscipline_df.sum().sort_values(ascending=False)

print("\nTop Disciplines by Total Count:")
print(discipline_totals)

print("\nTop Subdisciplines by Total Count:")
print(subdiscipline_totals)

# Save total counts
discipline_totals.to_csv("discipline_totals.csv")
subdiscipline_totals.to_csv("subdiscipline_totals.csv")

#  Save yearly trends
discipline_df.to_csv("discipline_yearly_trends.csv")
subdiscipline_df.to_csv("subdiscipline_yearly_trends.csv")


TOP 15 SUBDOMAINS

In [ ]:
import pandas as pd
import json
import re
from collections import defaultdict
import matplotlib.pyplot as plt

# -------------------------------
# 1. Load CSV File
# -------------------------------
csv_path = "results_combined_prompts_new.csv"
df = pd.read_csv(csv_path)

# -------------------------------
# 2. Safe JSON Parser
# -------------------------------
def safe_parse_json(cell):
    try:
        cell = re.sub(r"```json|```", "", str(cell)).strip()
        return json.loads(cell)
    except:
        return {}

# -------------------------------
# 3. Extract and Normalize Fields
# -------------------------------
discipline_year_counts = defaultdict(lambda: defaultdict(int))
subdiscipline_year_counts = defaultdict(lambda: defaultdict(int))

for _, row in df.iterrows():
    ont = safe_parse_json(row.get("ontology_classification", ""))
    year_entry = safe_parse_json(row.get("published_year", ""))
    year = year_entry.get("year", None)

    if year:
        try:
            year = int(year)
        except:
            continue

        discipline = ont.get("discipline", "").strip()
        subdiscipline = ont.get("subdiscipline", "").strip()

        if subdiscipline == "Smart/Cell phones":
            subdiscipline = "Small Scale Device Forensics"

        if discipline:
            discipline_year_counts[discipline][year] += 1
        if subdiscipline:
            subdiscipline_year_counts[subdiscipline][year] += 1

# -------------------------------
# 4. Convert to DataFrames
# -------------------------------
discipline_df = pd.DataFrame(discipline_year_counts).fillna(0).astype(int).sort_index()
subdiscipline_df = pd.DataFrame(subdiscipline_year_counts).fillna(0).astype(int).sort_index()

# -------------------------------
# 5. Plot Line Graph: Discipline Trends (unchanged)
# -------------------------------
plt.figure(figsize=(14, 6))
discipline_df.rolling(window=2, min_periods=1).mean().plot()
plt.title("Discipline Trends in DFRWS (2002–2025)")
plt.xlabel("Year")
plt.ylabel("Number of Papers")
plt.legend(title="Discipline", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# -------------------------------
# 6. Plot Line Graph: Subdiscipline Trends (unchanged)
# -------------------------------
plt.figure(figsize=(14, 6))
subdiscipline_df.rolling(window=2, min_periods=1).mean().plot()
plt.title("Subdiscipline Trends in DFRWS (2002–2025)")
plt.xlabel("Year")
plt.ylabel("Number of Papers")
plt.legend(title="Subdiscipline", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# -------------------------------
# 7. Save ONLY one CSV: Yearly trend for Top 15 subdisciplines
# -------------------------------
subdiscipline_totals = subdiscipline_df.sum().sort_values(ascending=False)
top15_subs = subdiscipline_totals.head(15).index.tolist()

sub_top15 = subdiscipline_df[top15_subs].copy()
sub_top15.index.name = "year"
sub_top15.to_csv("subdiscipline_yearly_trends_top15.csv")
print("Saved: subdiscipline_yearly_trends_top15.csv")


Domain Analysis

In [ ]:
# forensics_trends.py
import json
import re
import sys
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt


# --------------------- Config ---------------------
CSV_PATH = Path("results_combined_prompts_new.csv")  # input
PLOT_PATH = Path("discipline_trends_all.png")

CSV_DISC_TOTALS = Path("all_disciplines_2002_2025.csv")
CSV_SUB_TOTALS = Path("all_subdisciplines_2002_2025.csv")
CSV_DISC_MATRIX = Path("discipline_by_year_with_totals.csv")
CSV_SUB_MATRIX = Path("subdiscipline_by_year_with_totals.csv")


# ----------------- Parsing Helpers ----------------
def strip_code_fences(s: str) -> str:
    """Remove ```json ... ``` or bare ``` fences (and extra whitespace)."""
    if pd.isna(s):
        return ""
    s = str(s).strip()
    # remove starting ```json or ```
    s = re.sub(r"^\s*```json\s*", "", s, flags=re.IGNORECASE)
    s = re.sub(r"^\s*```\s*", "", s)
    # remove trailing ```
    s = re.sub(r"\s*```\s*$", "", s)
    return s.strip()


def parse_year(cell: str):
    """Return int year or None."""
    try:
        j = json.loads(strip_code_fences(cell))
        y = j.get("year")
        if y in (None, "", "Unknown"):
            return None
        return int(str(y))
    except Exception:
        return None


def parse_ontology(cell: str):
    """Return (discipline, subdiscipline). If missing, return empty strings."""
    try:
        j = json.loads(strip_code_fences(cell))
        return j.get("discipline", "") or "", j.get("subdiscipline", "") or ""
    except Exception:
        return "", ""


# -------------------- Load & Clean -----------------
def load_clean_df(csv_path: Path) -> pd.DataFrame:
    raw = pd.read_csv(csv_path, dtype=str, keep_default_na=False)
    # normalize header names
    raw.columns = raw.columns.str.strip().str.lower()

    # parse fields safely
    year = raw.get("published_year", "").apply(parse_year)
    disc_sub = raw.get("ontology_classification", "").apply(parse_ontology)
    discipline = disc_sub.apply(lambda t: t[0]).astype(str).str.strip()
    subdiscipline = disc_sub.apply(lambda t: t[1]).astype(str).str.strip()

    df = pd.DataFrame(
        {
            "year": year,
            "discipline": discipline.replace({"None": "", "null": ""}),
            "subdiscipline": subdiscipline.replace({"None": "", "null": ""}),
        }
    )
    # unify blanks as "Unknown"
    df["discipline"] = df["discipline"].replace("", "Unknown")
    df["subdiscipline"] = df["subdiscipline"].replace("", "Unknown")
    # drop rows without year and cast to int
    df = df.dropna(subset=["year"]).astype({"year": int})

    return df


# -------- ALL disciplines/subdisciplines with totals --------
def by_year_counts(df: pd.DataFrame, column: str):
    out = {}
    for yr, grp in df.groupby("year", sort=True):
        counts = grp[column].value_counts(dropna=False).sort_index()
        total = int(counts.sum())
        out[yr] = {
            "rows": [(name, int(cnt), (cnt / total) * 100) for name, cnt in counts.items()],
            "total": total,
        }
    return out


def print_section_with_totals(title: str, year_map: dict):
    print(f"\n{title}")
    grand = 0
    for yr in sorted(year_map):
        print(f"\nYear: {yr}")
        for name, cnt, pct in year_map[yr]["rows"]:
            print(f"  {name}: {cnt} ({pct:.2f}%)")
        print(f"  TOTAL: {year_map[yr]['total']}")
        grand += year_map[yr]["total"]
    return grand


# -------------------- Plotting ---------------------
def plot_all_disciplines(df: pd.DataFrame, out_path: Path):
    """
    Line chart of ALL disciplines by year.
    (If there are many lines, the legend is placed outside to reduce clutter.)
    """
    # matrix: rows=year, cols=discipline; zeros filled
    mat = pd.crosstab(df["year"], df["discipline"]).sort_index()

    # make the plot
    plt.figure(figsize=(14, 7))
    for col in mat.columns:
        plt.plot(mat.index, mat[col], marker="o", linewidth=1.8, label=col)

    plt.title("Discipline Trends by Year (All Disciplines)")
    plt.xlabel("Year")
    plt.ylabel("Count")
    plt.grid(True, alpha=0.3)
    # legend outside right; may be long, but you asked for all
    plt.legend(title="Discipline", bbox_to_anchor=(1.02, 1), loc="upper left", borderaxespad=0.)
    plt.tight_layout()
    plt.savefig(out_path, dpi=200)
    plt.close()


# ------------------------- Main -------------------------
def main():
    if not CSV_PATH.exists():
        print(f"ERROR: '{CSV_PATH}' not found.")
        sys.exit(1)

    df = load_clean_df(CSV_PATH)
    print(f"Rows loaded (non-empty year): {len(df)}")
    print("Columns:", df.columns.tolist())

    # Per-year maps
    disc_year = by_year_counts(df, "discipline")
    sub_year = by_year_counts(df, "subdiscipline")

    # Print sections with totals
    gdisc = print_section_with_totals("Discipline trends (ALL values):", disc_year)
    gsub = print_section_with_totals("Subdiscipline trends (ALL values):", sub_year)

    # Final overall totals
    disc_totals = df["discipline"].value_counts().reset_index()
    disc_totals.columns = ["Discipline", "Count"]
    sub_totals = df["subdiscipline"].value_counts().reset_index()
    sub_totals.columns = ["Subdiscipline", "Count"]

    print("\n=== FINAL TOTALS ===")
    print(f"Rows used: {len(df)}")
    print(f"Sum of per-year discipline totals:   {gdisc}")
    print(f"Sum of per-year subdiscipline totals: {gsub}")

    print("\nOverall discipline totals:")
    for _, r in disc_totals.iterrows():
        print(f"  {r['Discipline']}: {int(r['Count'])}")
    print(f"  GRAND TOTAL: {int(disc_totals['Count'].sum())}")

    print("\nOverall subdiscipline totals:")
    for _, r in sub_totals.iterrows():
        print(f"  {r['Subdiscipline']}: {int(r['Count'])}")
    print(f"  GRAND TOTAL: {int(sub_totals['Count'].sum())}")

    # Wide matrices with row/column totals
    disc_matrix = pd.crosstab(df["year"], df["discipline"]).sort_index()
    disc_matrix["TOTAL"] = disc_matrix.sum(axis=1)
    disc_matrix.loc["GRAND_TOTAL"] = disc_matrix.sum(axis=0)

    sub_matrix = pd.crosstab(df["year"], df["subdiscipline"]).sort_index()
    sub_matrix["TOTAL"] = sub_matrix.sum(axis=1)
    sub_matrix.loc["GRAND_TOTAL"] = sub_matrix.sum(axis=0)

    # Save CSVs
    disc_totals.to_csv(CSV_DISC_TOTALS, index=False)
    sub_totals.to_csv(CSV_SUB_TOTALS, index=False)
    disc_matrix.to_csv(CSV_DISC_MATRIX)
    sub_matrix.to_csv(CSV_SUB_MATRIX)

    # Plot ALL disciplines over years
    plot_all_disciplines(df, PLOT_PATH)

    print("\nSaved files:")
    print(f"  {CSV_DISC_TOTALS}")
    print(f"  {CSV_SUB_TOTALS}")
    print(f"  {CSV_DISC_MATRIX}")
    print(f"  {CSV_SUB_MATRIX}")
    print(f"  {PLOT_PATH}")


if __name__ == "__main__":
    main()


In [ ]:
import json
import re
import pandas as pd
import matplotlib.pyplot as plt

# ---------- CONFIG ----------
INPUT_CSV = "results_combined_prompts_new.csv"
YEAR_MIN, YEAR_MAX = 2002, 2025   # change if needed

# ---------- helpers ----------
FENCE_RE = re.compile(r"^```json\s*|\s*```$", re.IGNORECASE)

def parse_fenced_json(cell, default=None):
    """
    Accepts strings like:
      "```json\n{ \"year\": \"2007\" }\n```"
      or plain JSON, or already-dict.
    Returns dict or `default`.
    """
    if isinstance(cell, dict):
        return cell
    if not isinstance(cell, str) or not cell.strip():
        return default
    s = FENCE_RE.sub("", cell.strip())  # drop ```json ... ```
    try:
        return json.loads(s)
    except Exception:
        return default

def coerce_year(y):
    """Return int year if valid, else None."""
    try:
        y = int(str(y).strip())
        return y if (YEAR_MIN <= y <= YEAR_MAX) else None
    except Exception:
        return None

# ---------- load & parse ----------
raw = pd.read_csv(INPUT_CSV)

# Parse published_year -> year (int)
pub_parsed = raw["published_year"].apply(parse_fenced_json, default={})
years = pub_parsed.apply(lambda d: (d or {}).get("year"))
years = years.apply(coerce_year)

# Parse ontology_classification -> discipline/subdiscipline
onto_parsed = raw["ontology_classification"].apply(parse_fenced_json, default={})
discipline = onto_parsed.apply(lambda d: (d or {}).get("discipline", "")).fillna("")
subdiscipline = onto_parsed.apply(lambda d: (d or {}).get("subdiscipline", "")).fillna("")

# Build the cleaned DF
df = pd.DataFrame({
    "year": years,
    "discipline": discipline.astype(str).str.strip(),
    "subdiscipline": subdiscipline.astype(str).str.strip(),
}).dropna(subset=["year"])

df["year"] = df["year"].astype(int)

# ---------- discipline trend matrix (ALL disciplines, zeros filled) ----------
counts = (
    df.groupby(["year", "discipline"])
      .size()
      .reset_index(name="count")
)

all_years = list(range(YEAR_MIN, YEAR_MAX + 1))
mat = (
    counts.pivot(index="year", columns="discipline", values="count")
          .reindex(all_years, fill_value=0)
)

# make sure matrix is all integer zeros (no NaN)
mat = mat.fillna(0).astype(int)

# ---------- export matrix used for the line chart (zeros in CSV) ----------
mat.to_csv("discipline_trend_by_year_all.csv", na_rep="0")
print("Saved: discipline_trend_by_year_all.csv")

# ---------- “with totals” wide table ----------
disc_matrix = pd.crosstab(df["year"], df["discipline"]).sort_index()
# guarantee year rows in range even if no papers that year
disc_matrix = disc_matrix.reindex(index=all_years, fill_value=0)
disc_matrix["TOTAL"] = disc_matrix.sum(axis=1)
disc_matrix.loc["GRAND_TOTAL"] = disc_matrix.sum(axis=0)

disc_matrix = disc_matrix.fillna(0).astype(int)
disc_matrix.to_csv("discipline_by_year_with_totals.csv", na_rep="0")
print("Saved: discipline_by_year_with_totals.csv")

# ---------- plot ALL discipline lines ----------
# (Order columns so high-volume lines are drawn last and remain visible)
order = mat.sum(axis=0).sort_values(ascending=True).index
mat = mat[order]

plt.figure(figsize=(14, 7))
for col in mat.columns:
    plt.plot(mat.index, mat[col], marker="o", linewidth=1.8, alpha=0.9, label=col)

plt.title("DFRWS Disciplines by Year (All Disciplines)")
plt.xlabel("Year")
plt.ylabel("Count")
plt.xticks(all_years, rotation=45)
plt.grid(True, linestyle="--", alpha=0.35)
plt.legend(
    title="Discipline",
    fontsize=8,
    title_fontsize=9,
    loc="center left",
    bbox_to_anchor=(1.02, 0.5),
    frameon=False,
    ncol=1
)
plt.tight_layout()
plt.show()


Key Domains , All papers and CDF

In [ ]:
import pandas as pd, json, re
import matplotlib.pyplot as plt

# ---------- config ----------
INPUT_CSV = "results_combined_prompts_new.csv"
YEAR_MIN, YEAR_MAX = 2002, 2025
all_years = list(range(YEAR_MIN, YEAR_MAX + 1))

# ---------- robust parsers ----------
def strip_fences(s: str) -> str:
    if not isinstance(s, str): return ""
    # remove ANY ``` or ```json occurrences, anywhere in the cell
    s = re.sub(r"```(?:json)?", "", s, flags=re.IGNORECASE)
    s = s.replace("```", "")
    return s.strip()

def parse_json_messy(cell, default=None):
    if isinstance(cell, dict):
        return cell
    s = strip_fences(cell)
    try:
        return json.loads(s)
    except Exception:
        # fallback: pull "year": "2007" with regex if JSON is broken
        m = re.search(r'"year"\s*:\s*"?(?P<y>\d{4})"?', s or "")
        if m:
            return {"year": m.group("y")}
        return default if default is not None else {}

def coerce_year(y):
    try:
        y = int(str(y).strip())
        return y if YEAR_MIN <= y <= YEAR_MAX else None
    except Exception:
        return None

# ---------- rebuild df (year + discipline) ----------
raw = pd.read_csv(INPUT_CSV)

years = raw.get("published_year", pd.Series([""] * len(raw))).apply(parse_json_messy, default={})
years = years.apply(lambda d: (d or {}).get("year"))
years = years.apply(coerce_year)

onto = raw.get("ontology_classification", pd.Series([""] * len(raw))).apply(parse_json_messy, default={})
discipline = onto.apply(lambda d: (d or {}).get("discipline", "")).fillna("").astype(str).str.strip()

df = pd.DataFrame({"year": years, "discipline": discipline}).dropna(subset=["year"])
df["year"] = df["year"].astype(int)

# ---------- year × discipline matrix (zeros for missing years) ----------
counts = (
    df.groupby(["year", "discipline"])
      .size()
      .reset_index(name="count")
)
mat = (
    counts.pivot(index="year", columns="discipline", values="count")
          .reindex(all_years, fill_value=0)
          .fillna(0).astype(int)
)

# ---------- CDF-style: ALL papers + Top-5 disciplines ----------
# ALL papers per year
yearly_all = df.groupby("year").size().reindex(all_years, fill_value=0)

# Top-5 disciplines by total volume
top5 = mat.sum(axis=0).sort_values(ascending=False).head(6).index.tolist()
print("Top-5 disciplines:", top5)

# Yearly counts table for ALL + Top-5
cdf_counts = pd.DataFrame({"ALL Papers": yearly_all})
for d in top5:
    cdf_counts[d] = mat[d].reindex(all_years, fill_value=0)

# Cumulative sums (CDF curves)
cdf_cum = cdf_counts.cumsum()

# Plot cumulative counts
plt.figure(figsize=(14, 7))
for col in cdf_cum.columns:
    plt.plot(cdf_cum.index, cdf_cum[col], linewidth=2.2, label=col)
plt.title("Cumulative Papers by Year (ALL + Top-6 Disciplines)")
plt.xlabel("Year"); plt.ylabel("Cumulative Count")
plt.xticks(all_years, rotation=45); plt.grid(True, linestyle="--", alpha=0.35)
plt.legend(title="Series", loc="center left", bbox_to_anchor=(1.02, 0.5), frameon=False)
plt.tight_layout(); plt.show()

# (Optional) Normalized CDFs (0..1) to compare timing of growth
cdf_norm = cdf_cum.div(cdf_cum.iloc[-1])
plt.figure(figsize=(14, 7))
for col in cdf_norm.columns:
    plt.plot(cdf_norm.index, cdf_norm[col], linewidth=2.0, label=col)
plt.title("Normalized CDF (Proportion Reached by Year)")
plt.xlabel("Year"); plt.ylabel("Proportion of Final Total"); plt.ylim(0, 1.02)
plt.xticks(all_years, rotation=45); plt.grid(True, linestyle="--", alpha=0.35)
plt.legend(title="Series", loc="center left", bbox_to_anchor=(1.02, 0.5), frameon=False)
plt.tight_layout(); plt.show()

# Exports
cdf_counts.to_csv("cdf_all_plus_top6_yearly_counts.csv")
cdf_cum.to_csv("cdf_all_plus_top6_cumulative_counts.csv")
cdf_norm.to_csv("cdf_all_plus_top6_normalized_cdf.csv")
print("Saved CDF CSVs.")


SCHOOL AFFILIATION(ALL)

In [ ]:
import pandas as pd
import json
import re
from collections import Counter


csv_path = "results_combined_prompts_new.csv"  # Ensure this file is in the same folder
year_min, year_max = 2001, 2025               # Inclusive range

# Helpers
def safe_parse_json(cell):
    """
    Parse cells that should contain JSON.
    Strips code fences like ```json ... ``` if present.
    Returns {} on failure.
    """
    try:
        cleaned = re.sub(r"```json|```", "", str(cell)).strip()
        return json.loads(cleaned)
    except Exception:
        return {}

def is_year_in_range(entry, lo=year_min, hi=year_max):
    try:
        year = int(entry.get("year", "0"))
        return lo <= year <= hi
    except Exception:
        return False

def is_dfrws_any(entry):
    """
    Match ALL DFRWS venues (USA, EU, APAC, etc.)
    Expects parsed conference like {"conference": "DFRWS USA"}.
    """
    conf = str(entry.get("conference", "")).strip()
    return conf.startswith("DFRWS")


# Load
try:
    df = pd.read_csv(csv_path)
except FileNotFoundError:
    print("ERROR: 'results_combined_prompts_new.csv' not found.")
    raise SystemExit(1)

# Parse columns expected to be JSON blobs
df["school_names_parsed"] = df["school/organization_names"].apply(safe_parse_json)
df["authors_parsed"]      = df["authors"].apply(safe_parse_json)
df["countries_parsed"]    = df["author_countries"].apply(safe_parse_json)
df["year_parsed"]         = df["published_year"].apply(safe_parse_json)
df["conference_parsed"]   = df["conference"].apply(safe_parse_json)

# Filter: ALL DFRWS venues + year range (USA filter removed)
mask = (
    df["conference_parsed"].apply(is_dfrws_any) &
    df["year_parsed"].apply(is_year_in_range)
)
filtered_df = df[mask].copy()

# Counters
school_counter     = Counter()
author_counter     = Counter()
country_counter    = Counter()
conference_counter = Counter()   # e.g., DFRWS USA vs DFRWS EU, etc.

for _, row in filtered_df.iterrows():
    # --- Conferences (string) ---
    conf = str(row["conference_parsed"].get("conference", "")).strip()
    if conf:
        conference_counter[conf] += 1

    # --- Schools (list) ---
    schools = row["school_names_parsed"].get("school_names", [])
    for school in schools:
        s = str(school).strip()
        if s:
            school_counter[s] += 1

    # --- Authors (comma-separated string inside JSON) ---
    authors_raw = row["authors_parsed"].get("authors", "")
    authors = [a.strip() for a in str(authors_raw).split(",") if a.strip()]
    for author in authors:
        author_counter[author] += 1

    # --- Countries (list) ---
    countries = row["countries_parsed"].get("author_countries", [])
    for c in countries:
        cc = str(c).strip()
        if cc:
            country_counter[cc] += 1

# DataFrames
top_schools_df = (
    pd.DataFrame(school_counter.items(), columns=["School", "Count"])
    .sort_values(by="Count", ascending=False)
    .reset_index(drop=True)
)
top_authors_df = (
    pd.DataFrame(author_counter.items(), columns=["Author", "Count"])
    .sort_values(by="Count", ascending=False)
    .reset_index(drop=True)
)
top_countries_df = (
    pd.DataFrame(country_counter.items(), columns=["Country", "Count"])
    .sort_values(by="Count", ascending=False)
    .reset_index(drop=True)
)
conference_df = (
    pd.DataFrame(conference_counter.items(), columns=["Conference", "PaperCount"])
    .sort_values(by="PaperCount", ascending=False)
    .reset_index(drop=True)
)


print("\n=== DFRWS Venue Breakdown (2002–2025) ===")
for _, r in conference_df.iterrows():
    print(f"{r['Conference']}: {r['PaperCount']} papers")

print("\n=== Top School Affiliations (DFRWS, 2002–2025) ===")
for _, r in top_schools_df.iterrows():
    print(f"{r['School']}: {r['Count']}")

print("\n=== Most Frequent Authors (DFRWS, 2002–2025) ===")
for _, r in top_authors_df.iterrows():
    print(f"{r['Author']}: {r['Count']}")

print("\n=== Countries in Author Affiliations (DFRWS, 2002–2025) ===")
for _, r in top_countries_df.iterrows():
    print(f"{r['Country']}: {r['Count']}")

print(f"\n📄 Total Number of Matching Papers: {len(filtered_df)}")

# Save CSVs (neutral filenames)
top_schools_df.to_csv("top_schools_dfrws_2002_2025.csv", index=False)
top_authors_df.to_csv("top_authors_dfrws_2002_2025.csv", index=False)
top_countries_df.to_csv("top_countries_dfrws_2002_2025.csv", index=False)
conference_df.to_csv("dfrws_venue_breakdown_2002_2025.csv", index=False)


In [ ]:
import pandas as pd
import json
import re
from collections import Counter

# Config
csv_path = "results_combined_prompts_new.csv"  # Ensure this file is in the same folder
year_min, year_max = 2001, 2025               # Inclusive range
TOP_N = 10                                    # <-- Top 10 only

# Helpers
def safe_parse_json(cell):
    """
    Parse cells that should contain JSON.
    Strips code fences like ```json ... ``` if present.
    Returns {} on failure.
    """
    try:
        cleaned = re.sub(r"```json|```", "", str(cell)).strip()
        return json.loads(cleaned)
    except Exception:
        return {}

def is_year_in_range(entry, lo=year_min, hi=year_max):
    try:
        year = int(entry.get("year", "0"))
        return lo <= year <= hi
    except Exception:
        return False

def is_dfrws_any(entry):
    """
    Match ALL DFRWS venues (USA, EU, APAC, etc.)
    Expects parsed conference like {"conference": "DFRWS USA"}.
    """
    conf = str(entry.get("conference", "")).strip()
    return conf.startswith("DFRWS")

# Load
try:
    df = pd.read_csv(csv_path)
except FileNotFoundError:
    print("ERROR: 'results_combined_prompts_new.csv' not found.")
    raise SystemExit(1)

# Parse columns expected to be JSON blobs
df["school_names_parsed"] = df["school/organization_names"].apply(safe_parse_json)
df["authors_parsed"]      = df["authors"].apply(safe_parse_json)
df["countries_parsed"]    = df["author_countries"].apply(safe_parse_json)
df["year_parsed"]         = df["published_year"].apply(safe_parse_json)
df["conference_parsed"]   = df["conference"].apply(safe_parse_json)

# Filter: ALL DFRWS venues + year range (no USA-only filter)
mask = (
    df["conference_parsed"].apply(is_dfrws_any) &
    df["year_parsed"].apply(is_year_in_range)
)
filtered_df = df[mask].copy()

# Counters
school_counter     = Counter()
author_counter     = Counter()
country_counter    = Counter()
conference_counter = Counter()   # e.g., DFRWS USA vs DFRWS EU, etc.

for _, row in filtered_df.iterrows():
    # --- Conferences (string) ---
    conf = str(row["conference_parsed"].get("conference", "")).strip()
    if conf:
        conference_counter[conf] += 1

    # --- Schools (list) ---
    schools = row["school_names_parsed"].get("school_names", [])
    for school in schools:
        s = str(school).strip()
        if s:
            school_counter[s] += 1

    # --- Authors (comma-separated string inside JSON) ---
    authors_raw = row["authors_parsed"].get("authors", "")
    authors = [a.strip() for a in str(authors_raw).split(",") if a.strip()]
    for author in authors:
        author_counter[author] += 1

    # --- Countries (list) ---
    countries = row["countries_parsed"].get("author_countries", [])
    for c in countries:
        cc = str(c).strip()
        if cc:
            country_counter[cc] += 1

# DataFrames (full), then restrict to Top N
schools_df_all = (
    pd.DataFrame(school_counter.items(), columns=["School", "Count"])
    .sort_values(by="Count", ascending=False)
    .reset_index(drop=True)
)
authors_df_all = (
    pd.DataFrame(author_counter.items(), columns=["Author", "Count"])
    .sort_values(by="Count", ascending=False)
    .reset_index(drop=True)
)
countries_df_all = (
    pd.DataFrame(country_counter.items(), columns=["Country", "Count"])
    .sort_values(by="Count", ascending=False)
    .reset_index(drop=True)
)
conference_df_all = (
    pd.DataFrame(conference_counter.items(), columns=["Conference", "PaperCount"])
    .sort_values(by="PaperCount", ascending=False)
    .reset_index(drop=True)
)

top_schools_df   = schools_df_all.head(TOP_N)
top_authors_df   = authors_df_all.head(TOP_N)
top_countries_df = countries_df_all.head(TOP_N)
top_conference_df = conference_df_all.head(TOP_N)

# Output (formatted to avoid "USA4" run-ons)
# 
print(f"\n=== DFRWS Venue Breakdown (2002–2025) — Top {TOP_N} ===")
for _, r in top_conference_df.iterrows():
    print(f"{r['Conference']}: {r['PaperCount']} papers")

print(f"\n=== Top {TOP_N} School Affiliations (DFRWS, 2002–2025) ===")
for _, r in top_schools_df.iterrows():
    print(f"{r['School']}: {r['Count']}")

print(f"\n=== Top {TOP_N} Most Frequent Authors (DFRWS, 2002–2025) ===")
for _, r in top_authors_df.iterrows():
    print(f"{r['Author']}: {r['Count']}")

print(f"\n=== Top {TOP_N} Countries in Author Affiliations (DFRWS, 2002–2025) ===")
for _, r in top_countries_df.iterrows():
    print(f"{r['Country']}: {r['Count']}")

print(f"\n Total Number of Matching Papers: {len(filtered_df)}")

# Save CSVs (Top N only)
top_schools_df.to_csv(f"top_{TOP_N}_schools_dfrws_2002_2025.csv", index=False)
top_authors_df.to_csv(f"top_{TOP_N}_authors_dfrws_2002_2025.csv", index=False)
top_countries_df.to_csv(f"top_{TOP_N}_countries_dfrws_2002_2025.csv", index=False)
top_conference_df.to_csv(f"top_{TOP_N}_dfrws_venue_breakdown_2002_2025.csv", index=False)


In [ ]:
"""
DFRWS ANALYSIS — ONE GROUPED+STACKED GRAPH (Top-10 global countries + Other)
+ prints/saves the per-year LONG table for LaTeX
+ Top-10 tables + Per-conference Top-10 (with Other)
"""

import os
import re
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter


# Config

CSV_PATH = "results_combined_prompts_new.csv"
OUTDIR = "dfrws_outputs_grouped"
YEAR_MIN, YEAR_MAX = 2002, 2025

TOP_N_LISTS = 10              # Top-N for schools/authors/countries tables
TOP_N_COUNTRIES_GLOBAL = 10   # EXACTLY top-10 countries in stacks; others -> "Other"

os.makedirs(OUTDIR, exist_ok=True)


# Helpers

def safe_parse_json(cell):
    try:
        cleaned = re.sub(r"```json|```", "", str(cell)).strip()
        return json.loads(cleaned)
    except Exception:
        return {}

def is_year_in_range(entry, lo=YEAR_MIN, hi=YEAR_MAX):
    try:
        year = int(entry.get("year", "0"))
        return lo <= year <= hi
    except Exception:
        return False

def is_dfrws_any(entry):
    conf = str(entry.get("conference", "")).strip()
    return conf.startswith("DFRWS")

# Normalize country names to avoid splitting (UK vs United Kingdom, etc.)
ALIASES = {
    # USA
    "united states": "USA", "united states of america": "USA",
    "u.s.": "USA", "u.s.a.": "USA", "us": "USA", "u.s": "USA", "usa": "USA",
    # UK
    "uk": "United Kingdom", "u.k.": "United Kingdom",
    "england": "United Kingdom", "scotland": "United Kingdom",
    "wales": "United Kingdom", "great britain": "United Kingdom",
    "united kingdom": "United Kingdom",
    # South Korea
    "republic of korea": "South Korea",
    "korea, republic of": "South Korea",
    "korea (south)": "South Korea",
    # ISO-2 shortcuts & variants
    "de": "Germany", "gb": "United Kingdom", "kr": "South Korea",
    "ch": "Switzerland", "ie": "Ireland", "ca": "Canada",
    "au": "Australia", "nl": "Netherlands",
    "the netherlands": "Netherlands",
    "deutschland": "Germany",
}

def canon_country(c):
    key = str(c).strip().lower()
    return ALIASES.get(key, str(c).strip())

def sort_confs(confs):
    def key_fn(s):
        s_low = s.lower()
        if "usa" in s_low:  return (0, s_low)
        if "eu" in s_low:   return (1, s_low)
        if "apac" in s_low: return (2, s_low)
        return (3, s_low)
    return sorted(confs, key=key_fn)

def _safe_name(s):
    return re.sub(r"[^A-Za-z0-9_.-]+", "_", str(s))


# Load & Parse

try:
    df = pd.read_csv(CSV_PATH)
except FileNotFoundError:
    raise SystemExit(f"ERROR: '{CSV_PATH}' not found.")

df["school_names_parsed"] = df["school/organization_names"].apply(safe_parse_json)
df["authors_parsed"]      = df["authors"].apply(safe_parse_json)
df["countries_parsed"]    = df["author_countries"].apply(safe_parse_json)
df["year_parsed"]         = df["published_year"].apply(safe_parse_json)
df["conference_parsed"]   = df["conference"].apply(safe_parse_json)

mask = (
    df["conference_parsed"].apply(is_dfrws_any) &
    df["year_parsed"].apply(is_year_in_range)
)
filtered = df[mask].copy()


# Top lists + raw records

school_counter  = Counter()
author_counter  = Counter()
country_counter = Counter()

records = []  # (year, conference, country)

for _, row in filtered.iterrows():
    # schools
    for s in row["school_names_parsed"].get("school_names", []):
        s = str(s).strip()
        if s:
            school_counter[s] += 1
    # authors
    authors_raw = row["authors_parsed"].get("authors", "")
    for a in [x.strip() for x in str(authors_raw).split(",") if x.strip()]:
        author_counter[a] += 1
    # countries (+ capture conf/year)
    year = row["year_parsed"].get("year", None)
    try:
        year = int(year)
    except Exception:
        year = None
    conf = str(row["conference_parsed"].get("conference", "")).strip()
    for c in row["countries_parsed"].get("author_countries", []):
        cc = canon_country(c)
        if cc:
            country_counter[cc] += 1
            if year is not None and conf:
                records.append((year, conf, cc))

# Save Top-10 CSVs
pd.DataFrame(school_counter.items(), columns=["School","Count"]) \
  .sort_values("Count", ascending=False).head(TOP_N_LISTS) \
  .to_csv(os.path.join(OUTDIR, f"top_{TOP_N_LISTS}_schools_dfrws_{YEAR_MIN}_{YEAR_MAX}.csv"), index=False)

pd.DataFrame(author_counter.items(), columns=["Author","Count"]) \
  .sort_values("Count", ascending=False).head(TOP_N_LISTS) \
  .to_csv(os.path.join(OUTDIR, f"top_{TOP_N_LISTS}_authors_dfrws_{YEAR_MIN}_{YEAR_MAX}.csv"), index=False)

pd.DataFrame(country_counter.items(), columns=["Country","Count"]) \
  .sort_values("Count", ascending=False).head(TOP_N_LISTS) \
  .to_csv(os.path.join(OUTDIR, f"top_{TOP_N_LISTS}_countries_dfrws_{YEAR_MIN}_{YEAR_MAX}.csv"), index=False)

print(f"Saved Top-{TOP_N_LISTS} tables to: {OUTDIR}")

# Build counts for grouped+stacked chart

if not records:
    raise SystemExit("No (year, conference, country) records found after filtering.")

counts = pd.DataFrame(records, columns=["year","conference","country"]) \
           .groupby(["year","conference","country"]).size().reset_index(name="count")

years = sorted([y for y in counts["year"].unique() if YEAR_MIN <= y <= YEAR_MAX])
confs = sort_confs(counts["conference"].unique())

# ----- EXACTLY Top-10 global countries; rest = "Other" -----
global_totals = counts.groupby("country")["count"].sum().sort_values(ascending=False)
named_countries = list(global_totals.head(TOP_N_COUNTRIES_GLOBAL).index)

counts["country"] = counts["country"].where(counts["country"].isin(named_countries), other="Other")
counts = counts.groupby(["year","conference","country"])["count"].sum().reset_index()

stack_cols = named_countries + (["Other"] if "Other" in counts["country"].values else [])

# Build per-conference pivot with consistent columns
by_conf = {}
for conf in confs:
    pivot = counts[counts["conference"] == conf].pivot(index="year", columns="country", values="count")
    pivot = pivot.reindex(years).fillna(0.0)
    for c in stack_cols:
        if c not in pivot.columns:
            pivot[c] = 0.0
    pivot = pivot[stack_cols]
    by_conf[conf] = pivot

# -------------------------
# SAVE & PRINT the per-year data for LaTeX
# -------------------------
counts_long = counts.sort_values(["year", "conference", "country"]).copy()
counts_long_path = os.path.join(OUTDIR, f"counts_top10_other_LONG_{YEAR_MIN}_{YEAR_MAX}.csv")
counts_long.to_csv(counts_long_path, index=False)

for conf, pivot in by_conf.items():
    out = pivot.reset_index()  # columns: year + each of the 10 countries + Other
    out_path = os.path.join(OUTDIR, f"{_safe_name(conf)}_year_by_country_top10_other_{YEAR_MIN}_{YEAR_MAX}.csv")
    out.to_csv(out_path, index=False)

pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", 1000)
print("\n DATA (per-year, after Top-10 + Other mapping) ")
print(counts_long.to_string(index=False))
print("\n[INFO] Country stack order (legend order in the figure):")
print(", ".join(stack_cols))
print(f"\n[WROTE] {counts_long_path}")
for conf in by_conf:
    print(f"[WROTE] {os.path.join(OUTDIR, f'{_safe_name(conf)}_year_by_country_top10_other_{YEAR_MIN}_{YEAR_MAX}.csv')}")

# Plot: ONE grouped+stacked figure

fig, ax = plt.subplots(figsize=(14, 6))  # single plot; default matplotlib colors

n_conf = len(confs)
group_width = 0.8
bar_w = group_width / max(n_conf, 1)

x = np.arange(len(years))  # year positions
country_handles = []

HATCHES = ["", "///", "\\\\", "xx", "...", "++", "oo", "**", "||", "--"]

for i, conf in enumerate(confs):
    x_offset = x - (group_width/2) + (i + 0.5)*bar_w
    bottom = np.zeros(len(years))
    first_conf_for_country_legend = (i == 0)

    for j, c in enumerate(stack_cols):
        vals = by_conf[conf][c].values
        bars = ax.bar(
            x_offset, vals, bottom=bottom, width=bar_w,
            label=(c if first_conf_for_country_legend else None),
            hatch=HATCHES[i % len(HATCHES)], edgecolor="black"
        )
        if first_conf_for_country_legend and j == 0:
            country_handles = []
        if first_conf_for_country_legend:
            country_handles.append(bars[0])
        bottom = bottom + vals

ax.set_xticks(x)
ax.set_xticklabels(years, rotation=45, ha="right")
ax.set_xlabel("Year")
ax.set_ylabel("Count")
ax.set_title(f"DFRWS Authors by Country per Year (Grouped by Conference; Top {TOP_N_COUNTRIES_GLOBAL} + Other)")

# Legends
country_labels = stack_cols
leg1 = ax.legend(country_handles, country_labels, loc="upper center",
                 bbox_to_anchor=(0.5, -0.12), ncol=min(6, len(country_labels)),
                 frameon=False, fontsize="small")

from matplotlib.patches import Patch
conf_patches = [Patch(facecolor="white", edgecolor="black",
                      hatch=HATCHES[i % len(HATCHES)], label=conf)
                for i, conf in enumerate(confs)]
leg2 = ax.legend(handles=conf_patches, loc="upper left", bbox_to_anchor=(0.0, 1.02),
                 ncol=min(4, len(conf_patches)), frameon=False, fontsize="small")
ax.add_artist(leg1)

fig.tight_layout()
one_graph_path = os.path.join(
    OUTDIR,
    f"grouped_stacked_countries_by_year_top{TOP_N_COUNTRIES_GLOBAL}_{YEAR_MIN}_{YEAR_MAX}.png"
)
fig.savefig(one_graph_path, dpi=300)
plt.close(fig)
print(f"[OK] Saved ONE grouped+stacked figure: {one_graph_path}")


# ALL conferences: Top-10 countries tables (LONG + WIDE) with "Other"

raw_counts = pd.DataFrame(records, columns=["year","conference","country"])
totals_by_conf_country = raw_counts.groupby(["conference", "country"]).size().reset_index(name="total_count")

per_conf_rows = []
for conf, g in totals_by_conf_country.groupby("conference"):
    g = g.sort_values("total_count", ascending=False).reset_index(drop=True)
    named = g.head(TOP_N_COUNTRIES_GLOBAL).copy()
    other_count = int(g["total_count"].iloc[TOP_N_COUNTRIES_GLOBAL:].sum()) if len(g) > TOP_N_COUNTRIES_GLOBAL else 0
    if other_count > 0:
        named = pd.concat([named, pd.DataFrame([{"conference": conf, "country": "Other", "total_count": other_count}])],
                          ignore_index=True)
    total_conf = int(g["total_count"].sum())
    named["rank"] = range(1, len(named) + 1)
    named["share"] = (named["total_count"] / total_conf).round(4) if total_conf > 0 else 0.0
    per_conf_rows.append(named)

top_long = pd.concat(per_conf_rows, ignore_index=True).sort_values(["conference", "rank"])

long_path = os.path.join(OUTDIR, f"top_countries_per_conference_LONG_top{TOP_N_COUNTRIES_GLOBAL}_{YEAR_MIN}_{YEAR_MAX}.csv")
top_long.to_csv(long_path, index=False)

rows = []
for conf, g in top_long.groupby("conference"):
    g = g.sort_values("rank")
    row = {"conference": conf}
    for i, r in enumerate(g.itertuples(index=False), start=1):
        row[f"country_{i}"] = r.country
        row[f"count_{i}"]   = int(r.total_count)
        row[f"share_{i}"]   = float(r.share)
    rows.append(row)
top_wide = pd.DataFrame(rows)

wide_path = os.path.join(OUTDIR, f"top_countries_per_conference_WIDE_top{TOP_N_COUNTRIES_GLOBAL}_{YEAR_MIN}_{YEAR_MAX}.csv")
top_wide.to_csv(wide_path, index=False)

print(f"[OK] Saved per-conference Top-{TOP_N_COUNTRIES_GLOBAL} country tables (LONG): {long_path}")
print(f"[OK] Saved per-conference Top-{TOP_N_COUNTRIES_GLOBAL} country tables (WIDE): {wide_path}")

print("\nDone.")


In [ ]:
import pandas as pd
import json
import re
from collections import Counter

# ======== CONFIG ========
CSV_PATH = "results_combined_prompts_new.csv"   # your file
TARGET_COL = "forensic_vs_anti"
# ========================

# Robust read: let pandas infer comma/tab, handle quoted newlines
df = pd.read_csv(
    CSV_PATH,
    sep=None,                # auto-detect delimiter
    engine="python",
    dtype=str,
    keep_default_na=False    # keep empty strings instead of NaN
)

# Clean column names only for safety (preserve spaces like "Paper Title")
df.columns = [c.replace("\u00A0", " ").strip() for c in df.columns]

if TARGET_COL not in df.columns:
    raise KeyError(f"Column '{TARGET_COL}' not found. Columns: {df.columns.tolist()}")

# --- helpers ---
CODE_FENCE_RE = re.compile(r"^\s*```(?:json)?\s*|\s*```\s*$", re.IGNORECASE)

def strip_code_fences(s: str) -> str:
    if not isinstance(s, str):
        return ""
    return CODE_FENCE_RE.sub("", s).strip()

def parse_json_cell(cell):
    raw = strip_code_fences(cell)
    if not raw:
        return {}
    try:
        return json.loads(raw)
    except Exception:
        return {}

def normalize_forensic_type(val):
    if not val:
        return None
    v = str(val).strip().lower()
    if v == "forensic" or v in {"digital forensic", "digital forensics"}:
        return "Digital Forensic"
    if v in {"anti-forensic", "antiforensic", "anti forensic"}:
        return "Anti-Forensic"
    return None  # treat anything else as unparseable/unknown

# --- parse + count ---
types = (
    df[TARGET_COL]
      .apply(parse_json_cell)
      .apply(lambda d: normalize_forensic_type(d.get("forensic_type")))
)

counts = Counter(t for t in types if t)

# Print EXACTLY what you asked for
print("Digital Forensic:", counts.get("Digital Forensic", 0))
print("Anti-Forensic:   ", counts.get("Anti-Forensic", 0))


In [ ]:
import pandas as pd
import json
import re
from collections import Counter  # (not strictly needed, but ok to keep)
import os

# ========= CONFIG =========
CSV_PATH = "results_combined_prompts_new.csv"
OUTDIR = "dfrws_outputs_dfvsaf_trends"
FILTER_DFRWS_ONLY = True   # set False if you want all venues
INCLUDE_UNKNOWN = False    # set True to keep unknown/missing DF/AF labels
MAKE_PLOT = False          # set True to also save a simple matplotlib plot
# =========================

os.makedirs(OUTDIR, exist_ok=True)

# --- helpers (same pattern as before) ---
CODE_FENCE_RE = re.compile(r"^\s*```(?:json)?\s*|\s*```\s*$", re.IGNORECASE)

def strip_code_fences(s: str) -> str:
    if not isinstance(s, str):
        return ""
    return CODE_FENCE_RE.sub("", s).strip()

def parse_json_cell(cell):
    raw = strip_code_fences(cell)
    if not raw:
        return {}
    try:
        return json.loads(raw)
    except Exception:
        return {}

def normalize_forensic_type(val):
    if not val:
        return None
    v = str(val).strip().lower()
    if v == "forensic" or v in {"digital forensic", "digital forensics"}:
        return "Digital Forensic"
    if v in {"anti-forensic", "antiforensic", "anti forensic"}:
        return "Anti-Forensic"
    return None  # treat anything else as unknown

def parse_year(cell):
    d = parse_json_cell(cell)
    y = d.get("year")
    try:
        return int(y)
    except Exception:
        return None

def parse_conf(cell):
    d = parse_json_cell(cell)
    return str(d.get("conference", "")).strip()

# --- load (let pandas infer delimiter; handles quoted newlines) ---
df = pd.read_csv(
    CSV_PATH, sep=None, engine="python", dtype=str, keep_default_na=False
)
df.columns = [c.replace("\u00A0", " ").strip() for c in df.columns]

need_cols = ["published_year", "forensic_vs_anti", "conference"]
for c in need_cols:
    if c not in df.columns:
        raise KeyError(f"Missing column '{c}'. Found: {df.columns.tolist()}")

# --- parse needed fields ---
df["year"] = df["published_year"].apply(parse_year)
df["type"] = df["forensic_vs_anti"].apply(lambda s: normalize_forensic_type(parse_json_cell(s).get("forensic_type")))
df["conf"] = df["conference"].apply(parse_conf)

# optional filter: only DFRWS venues
if FILTER_DFRWS_ONLY:
    df = df[df["conf"].str.startswith("DFRWS", na=False)].copy()

if not INCLUDE_UNKNOWN:
    df = df[df["type"].isin(["Digital Forensic", "Anti-Forensic"])].copy()

df = df[df["year"].notna()].copy()

if df.empty:
    raise SystemExit("No rows after filtering/parsing.")

# ---------- LONG table: year × type ----------
long = (
    df.groupby(["year", "type"])
      .size().reset_index(name="count")
      .sort_values(["year", "type"])
)
long_path = os.path.join(OUTDIR, "dfvsaf_by_year_LONG.csv")
long.to_csv(long_path, index=False)

# ---------- WIDE table (with shares) ----------
wide = long.pivot(index="year", columns="type", values="count").fillna(0).astype(int)
for col in ["Digital Forensic", "Anti-Forensic"]:
    if col not in wide.columns:
        wide[col] = 0
wide = wide[["Digital Forensic", "Anti-Forensic"]]
wide["Total"] = wide.sum(axis=1)
wide["Digital_Forensic_share"] = (wide["Digital Forensic"] / wide["Total"]).fillna(0).round(4)
wide["Anti-Forensic_share"] = (wide["Anti-Forensic"] / wide["Total"]).fillna(0).round(4)
wide_path = os.path.join(OUTDIR, "dfvsaf_by_year_WIDE.csv")
wide.reset_index().to_csv(wide_path, index=False)

# ---------- print to console ----------
pd.set_option("display.max_rows", None)
print("\n[DF vs AF by Year — LONG]")
print(long.to_string(index=False))
print("\n[DF vs AF by Year — WIDE]")
print(wide.reset_index().to_string(index=False))
print(f"\n[WROTE] {long_path}")
print(f"[WROTE] {wide_path}")

# ---------- optional plot ----------
if MAKE_PLOT:
    import matplotlib.pyplot as plt  # no seaborn
    fig, ax = plt.subplots(figsize=(10, 5))
    years = wide.index.tolist()
    ax.plot(years, wide["Digital Forensic"].tolist(), label="Digital Forensic")
    ax.plot(years, wide["Anti-Forensic"].tolist(), label="Anti-Forensic")
    ax.set_xlabel("Year")
    ax.set_ylabel("Count")
    ax.set_title("DF vs AF Counts by Year")
    ax.legend()
    fig.tight_layout()
    plot_path = os.path.join(OUTDIR, "dfvsaf_by_year_lines.png")
    fig.savefig(plot_path, dpi=300)
    plt.close(fig)
    print(f"[WROTE] {plot_path}")


In [ ]:
import pandas as pd
import json, re
import matplotlib.pyplot as plt

CSV_PATH = "results_combined_prompts_new.csv"  

# --- helpers ---
CODE_FENCE_RE = re.compile(r"^\s*```(?:json)?\s*|\s*```\s*$", re.IGNORECASE)
def strip_code_fences(s): return CODE_FENCE_RE.sub("", str(s)).strip()

def parse_json(cell):
    raw = strip_code_fences(cell)
    try: return json.loads(raw) if raw else {}
    except Exception: return {}

def norm_type(v):
    if not v: return None
    v = str(v).strip().lower()
    if v == "forensic" or v in {"digital forensic","digital forensics"}:
        return "Digital Forensic"
    if v in {"anti-forensic","antiforensic","anti forensic"}:
        return "Anti-Forensic"
    return None

def parse_year(cell):
    y = parse_json(cell).get("year")
    try: return int(y)
    except Exception: return None

def parse_conf(cell):
    return str(parse_json(cell).get("conference","")).strip()

# --- load & parse ---
df = pd.read_csv(CSV_PATH, sep=None, engine="python", dtype=str, keep_default_na=False)
df.columns = [c.replace("\u00A0"," ").strip() for c in df.columns]

df["year"] = df["published_year"].apply(parse_year)
df["type"] = df["forensic_vs_anti"].apply(lambda s: norm_type(parse_json(s).get("forensic_type")))
df["conf"] = df["conference"].apply(parse_conf)

# DFRWS only + drop unknowns/invalid years
df = df[df["conf"].str.startswith("DFRWS", na=False)]
df = df[df["type"].isin(["Digital Forensic","Anti-Forensic"])]
df = df[df["year"].notna()]

# --- LONG table (year, type, count) ---
df_long = (
    df.groupby(["year","type"])
      .size().reset_index(name="count")
      .sort_values(["year","type"])
)

# --- WIDE for plotting ---
wide = df_long.pivot(index="year", columns="type", values="count").fillna(0).astype(int)
for col in ["Digital Forensic","Anti-Forensic"]:
    if col not in wide.columns: wide[col] = 0
wide = wide.reindex(sorted(wide.index))
years = wide.index.astype(int).tolist()
dfc = wide["Digital Forensic"].to_numpy()
afc = wide["Anti-Forensic"].to_numpy()

# --------- Figure A: Grouped bars (counts) ---------
x = range(len(years)); w = 0.42
fig, ax = plt.subplots(figsize=(12,5))
ax.bar([i - w/2 for i in x], dfc, width=w, label="Digital Forensic")
ax.bar([i + w/2 for i in x], afc, width=w, label="Anti-Forensic")

# label tiny AF bars
for i, v in enumerate(afc):
    if v > 0:
        ax.text(i + w/2, v + 0.2, str(v), ha="center", va="bottom", fontsize=8)

ax.set_xticks(list(x))
ax.set_xticklabels(years, rotation=45, ha="right")
ax.set_ylabel("Paper count")
ax.set_title("DF vs AF papers by year (DFRWS)")
ax.legend()
fig.tight_layout()
plt.show()

# --------- Figure B (optional): 100% stacked (shares) ---------
wide["Total"] = (wide["Digital Forensic"] + wide["Anti-Forensic"]).replace(0, 1)
share_df = wide[["Digital Forensic","Anti-Forensic"]].div(wide["Total"], axis=0)

fig2, ax2 = plt.subplots(figsize=(12,5))
ax2.bar(years, share_df["Digital Forensic"].values, label="Digital Forensic")
ax2.bar(years, share_df["Anti-Forensic"].values,
        bottom=share_df["Digital Forensic"].values, label="Anti-Forensic")
ax2.set_xticks(list(range(len(years))))
ax2.set_xticklabels(years, rotation=45, ha="right")
ax2.set_ylabel("Share")
ax2.set_title("DF vs AF shares by year (DFRWS)")
ax2.legend()
fig2.tight_layout()
plt.show()

